In [ ]:
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.datasets import fetch_openml
import numpy as np
import matplotlib.pyplot as plt


In [ ]:
def fetch_mnist():
    mnist = fetch_openml('mnist_784' , version=1 , as_frame=False)
    X , y = mnist.data , mnist.target.astype(int)

    mask = (y == 0) | (y == 1)
    X_filtered , y_filtered = X[mask] , y[mask]

    shuffle_index = np.random.permutation(len(X_filtered))

    X_shuffled = X_filtered[shuffle_index]
    y_shuffled = y_filtered[shuffle_index]

    return X_shuffled , y_shuffled

def preprocess_image(x , n_components):
    scaler = StandardScaler()
    pca = PCA(n_components=n_components)

    x_scaled = scaler.fit_transform(x)
    x_pca = pca.fit_transform(x_scaled)

    x_pca_normalized = 2.0*np.pi*(x_pca - x_pca.min(axis=0)) / (x_pca.max(axis=0) - x_pca.min(axis=0))
    return x_pca_normalized

In [ ]:
from pennylane import numpy as np
import pennylane as qml

def create_qnn(n_layers , n_qubits):
    dev = qml.device("default.qubit")
    @qml.qnode(dev)

    def circuit(inputs , params):
        
        # Angle Encoding
        for q , feature in enumerate(inputs):
            qml.RX(feature , wires=q)

        # Setup Circuit
        for layer in range(n_layers):

            for qubit in range(n_qubits):
                qml.RX(params[layer][qubit][0] , wires=qubit)
                qml.RY(params[layer][qubit][1] , wires=qubit)
                qml.RZ(params[layer][qubit][2] , wires=qubit)

            for qubit in range(n_qubits):
                if qubit != n_qubits-1:
                    qml.CNOT(wires=[qubit , qubit+1])
                else:
                    qml.CNOT(wires=[qubit , 0])

                # Measure first qubit and return result
                result = qml.probs(wires=0)
                return result
            
        return circuit

In [ ]:
def cross_entropy(output , true):
    e = 1e-10
    loss = -np.log(output[true] + e)
    return loss
def cross_entropy_derivative(output , true):
    e = 1e-10
    derivative = -1 / (output[true] + e)
    return derivative

In [ ]:
def train_qnn(n_layers , n_qubits , x , y , epochs , alpha=0.005):
    circuit = create_qnn(n_layers=n_layers , n_qubits=n_qubits)
    params = np.random.uniform(0 , 0.01 , size=(n_layers , n_qubits , 3))

    for epoch in range(epochs):
        epoch_loss = 0
        correct_preds = 0

        for image , label in zip(x,y):

            out = circuit(image , params)
            loss = cross_entropy(out , label)

            # Tracking training
            epoch_loss += loss
            if np.argmax(out) == label:
                correct_preds += 1

            # Derivative Loss
            output_gradients = np.zeros_like(out)
            output_gradients[label] = cross_entropy_derivative(out , label)
            # Gradients based on += pi/2
            gradients = np.zeroes_like(params)

            # Backpropogation
            for l in range(n_layers):
                for q in range(n_qubits):
                    for g in range(3):

                        params_plus = params.copy()
                        params_plus[l,q,g] += np.pi/2
                        out_plus = circuit(image , params_plus)

                        params_minus = params.copy()
                        params_minus[l,q,g] -= np.pi/2
                        out_minus = circuit(image , params_minus)

                        psr_grad = (out_plus - out_minus)/2

                        gradients[l,q,g] = np.dot(output_gradients , psr_grad)

            # Update Params
            params -= alpha*gradients

        print(f"Epoch {epoch}")
        print(f"Loss {epoch_loss/len(x)}")
        print(f"Accuracy: {correct_preds/len(x) * 100}%")

In [ ]:
n_qubits = 4
n_layers = 2
x,y = fetch_mnist()
x = preprocess_image(x , n_qubits)
x,y = x[0:50] , y[0:50]
train_qnn(n_layers , n_qubits , x , y , 10)